# Test out inversion of syntheetic EDM data with VMOD

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from vmod.data import Edm

# Load synthetic GNSS data and make it in to synthetic EDM data

In [2]:
# Load GNSS
csvfile='./examples/dvd/gps/Synth_ENU_Low_Gaussian.csv'
xs,ys,uxs,uys,uzs=np.loadtxt(csvfile,skiprows=1,delimiter=',',unpack=True)
euxs=xs*0+1e-3
euys=np.copy(euxs)
euzs=np.copy(euxs)
names=[str(i).zfill(4) for i in range(len(xs))]

In [3]:
# Pick a few base benchmarks
basei = [0, 20, 420, 440]

# Make fake EDM data to all of the rest
orig_bm = np.zeros((len(xs)-4, 3))
tail_bm = np.zeros((len(xs)-4, 3))
delta = np.zeros(len(xs)-4)

c = 0
for i in range(len(xs)):
    if(i in basei):
        continue
    orig_bm[c, :] = [xs[basei[i%4]], ys[basei[i%4]], 0]
    tail_bm[c, :] = [xs[i], ys[i], 0]
    
    x0 = orig_bm[c, 0]
    x1 = tail_bm[c, 0]
    x0shift = (orig_bm[c, 0]+uxs[basei[i%4]])
    x1shift = (tail_bm[c, 0]+uxs[i])
    
    y0 = orig_bm[c, 1]
    y1 = tail_bm[c, 1]
    y0shift = (orig_bm[c, 1]+uys[basei[i%4]])
    y1shift = (tail_bm[c, 1]+uys[i])
    
    d0 = np.sqrt((x1-x0)**2 + (y1-y0)**2)
    d1 = np.sqrt((x1shift-x0shift)**2 + (y1shift-y0shift)**2)
    
    delta[c] = d1-d0
    
    c += 1

# Create data object

In [4]:
obs = Edm()

obs.add_xorigins(orig_bm[:,0])
obs.add_yorigins(orig_bm[:,1])
obs.add_zorigins(orig_bm[:,2])

obs.add_xends(tail_bm[:,0])
obs.add_yends(tail_bm[:,1])
obs.add_zends(tail_bm[:,2])

# TODO: Add in check for 1D array here
obs.add_deltas(delta)

# Create model object

In [5]:
from vmod.source import Mctigue
mct = Mctigue(obs)
deltasm=mct.forward([0,0,2.0e3,5e2,1e6],unravel=True)
print(deltasm.shape,delta.shape,obs.get_data().shape)

(437,) (437,) (437,)


# Create Inversion object

In [ ]:
mct.set_x0([0,0,2.0e3,5e2,1e6])
#Bounds for parameters
mct.set_bounds(low_bounds = [-10000,-10000,5e2,1e1,1e5], high_bounds = [10000,10000,1e4,5e3,1e7])

inv = vmod.inverse.Inverse(obs)
inv.register_source(mct)

In [ ]:
ans=inv.nlsq()

print(ans.x)

In [ ]:
inv.residual(inv.get_x0())

In [ ]:
#inv.obs.get_data()
inv.forward(inv.get_x0())